<a href="https://colab.research.google.com/github/jingjieyuan573-bite/Composite_Distribution_analysis/blob/main/Composite_VaR_ES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import numpy as np
from scipy.stats import norm, t
from scipy.integrate import quad
from scipy.optimize import brentq
from math import log
from scipy.stats import chi2


# ----------------------------
# 1. 定义 Skew-Normal 和 Skew-t PDF
# ----------------------------
def skew_normal_pdf(x, xi, omega, alpha):
    z = (x - xi) / omega
    phi_z = norm.pdf(z)
    Phi_alpha_z = norm.cdf(alpha * z)
    return (2 / omega) * phi_z * Phi_alpha_z

def skew_t_pdf(x, xi_t, omega_t, alpha_t, nu):
    z = (x - xi_t) / omega_t
    t_pdf = t.pdf(z, df=nu)
    a = alpha_t * z * np.sqrt((nu + 1) / (nu + z**2))
    t_cdf = t.cdf(a, df=nu + 1)
    return (2 / omega_t) * t_pdf * t_cdf

def skew_normal_cdf(x, xi, omega, alpha):
    return quad(lambda t: skew_normal_pdf(t, xi, omega, alpha), -np.inf, x)[0]

def skew_t_cdf(x, xi_t, omega_t, alpha_t, nu):
    return quad(lambda t: skew_t_pdf(t, xi_t, omega_t, alpha_t, nu), -np.inf, x)[0]

# ----------------------------
# 2. composite CDF（数值积分）
# ----------------------------
def composite_cdf_exact(x, xi, omega, alpha, xi_t, omega_t, alpha_t, nu, theta1, theta2, r1, r2):
    S_t_theta1 = skew_t_cdf(theta1, xi_t, omega_t, alpha_t, nu)
    S_t_theta2 = skew_t_cdf(theta2, xi_t, omega_t, alpha_t, nu)
    S_n_theta1 = skew_normal_cdf(theta1, xi, omega, alpha)
    S_n_theta2 = skew_normal_cdf(theta2, xi, omega, alpha)

    if x <= theta1:
        integral, _ = quad(lambda z: skew_t_pdf(z, xi_t, omega_t, alpha_t, nu), -np.inf, x)
        return r1 * integral / S_t_theta1
    elif theta1 < x < theta2:
        integral, _ = quad(lambda z: skew_normal_pdf(z, xi, omega, alpha), theta1, x)
        return r1 + (1 - r1 - r2) * integral / (S_n_theta2 - S_n_theta1)
    else:
        integral, _ = quad(lambda z: skew_t_pdf(z, xi_t, omega_t, alpha_t, nu), theta2, x)
        return (1 - r2) + r2 * integral / (1 - S_t_theta2)

# ----------------------------
# 3. 参数设置
# ----------------------------
theta1, theta2 = -1.0, 1.5
xi, omega, alpha = 0.0, 1.5, -0.02
xi_t, omega_t, alpha_t, nu = -0.2, 2.8, -0.03, 4.0

# 计算 r1, r2
numerator_r1 = skew_t_pdf(theta2, xi_t, omega_t, alpha_t, nu) * \
               skew_normal_pdf(theta1, xi, omega, alpha) * \
               skew_t_cdf(theta1, xi_t, omega_t, alpha_t, nu)
denominator_r1 = skew_t_pdf(theta1, xi_t, omega_t, alpha_t, nu) * skew_t_pdf(theta2, xi_t, omega_t, alpha_t, nu) * \
    (skew_normal_cdf(theta2, xi, omega, alpha) - skew_normal_cdf(theta1, xi, omega, alpha)) + \
    skew_t_pdf(theta2, xi_t, omega_t, alpha_t, nu) * skew_t_cdf(theta1, xi_t, omega_t, alpha_t, nu) * skew_normal_pdf(theta1, xi, omega, alpha) + \
    skew_t_pdf(theta1, xi_t, omega_t, alpha_t, nu) * skew_normal_pdf(theta2, xi, omega, alpha) * (1 - skew_t_cdf(theta2, xi_t, omega_t, alpha_t, nu))
r_1 = numerator_r1 / denominator_r1

numerator_r2 = skew_t_pdf(theta1, xi_t, omega_t, alpha_t, nu) * \
               (skew_normal_cdf(theta2, xi, omega, alpha) - skew_normal_cdf(theta1, xi, omega, alpha)) + \
               skew_t_cdf(theta1, xi_t, omega_t, alpha_t, nu) * skew_normal_pdf(theta1, xi, omega, alpha)
denominator_r2 = skew_t_cdf(theta1, xi_t, omega_t, alpha_t, nu) * skew_normal_pdf(theta1, xi, omega, alpha)
r_2 = 1 - r_1 * (numerator_r2 / denominator_r2)

# ----------------------------
# 4. 生成 composite 分布随机数（逆 CDF）
# ----------------------------
def sample_composite(n):
    u = np.random.uniform(0, 1, n)
    samples = np.zeros(n)
    for i in range(n):
        samples[i] = brentq(lambda x: composite_cdf_exact(x, xi, omega, alpha, xi_t, omega_t, alpha_t, nu,
                                                          theta1, theta2, r_1, r_2) - u[i],
                            -50, 50)
    return samples

np.random.seed(123)
returns = sample_composite(2083)

# ----------------------------
# 5. 计算 95% VaR
# ----------------------------
alpha_level = 0.05
var_95 = brentq(
    lambda x: composite_cdf_exact(x, xi, omega, alpha, xi_t, omega_t, alpha_t, nu,
                                  theta1, theta2, r_1, r_2) - alpha_level,
    -50, 50
)

# ----------------------------
# 6. 违约次数 & Hit rate
# ----------------------------
violations = returns < var_95
n_violations = violations.sum()
n_obs = len(returns)
hit_rate = n_violations / n_obs

# ----------------------------
# 7. Kupiec LR_uc
# ----------------------------
eps = 1e-16
p = np.clip(hit_rate, eps, 1-eps)

LR_uc = -2 * ((n_obs-n_violations)*log(1-alpha_level) + n_violations*log(alpha_level) -
              (n_obs-n_violations)*log(1-p) - n_violations*log(p))
p_value_uc = 1 - chi2.cdf(LR_uc, df=1)

# ----------------------------
# 8. Christoffersen 独立性检验
# ----------------------------
violations_int = violations.astype(int)
N00 = N01 = N10 = N11 = 0
for t in range(1, n_obs):
    if violations_int[t-1] == 0 and violations_int[t] == 0:
        N00 += 1
    elif violations_int[t-1] == 0 and violations_int[t] == 1:
        N01 += 1
    elif violations_int[t-1] == 1 and violations_int[t] == 0:
        N10 += 1
    else:
        N11 += 1

pi01 = N01 / (N00 + N01) if (N00 + N01) > 0 else 0
pi11 = N11 / (N10 + N11) if (N10 + N11) > 0 else 0
pi = (N01 + N11) / (N00 + N01 + N10 + N11)
pi01 = np.clip(pi01, eps, 1-eps)
pi11 = np.clip(pi11, eps, 1-eps)
pi = np.clip(pi, eps, 1-eps)
L_ind = 2 * ((N00*log(1-pi01) + N01*log(pi01) + N10*log(1-pi11) + N11*log(pi11)) -
             ((N00+N10)*log(1-pi) + (N01+N11)*log(pi)))
p_value_ind = 1 - chi2.cdf(L_ind, df=1)

# ----------------------------
# 9. 实际 ES
# ----------------------------
excess_losses = returns[returns < var_95]
es_realized = excess_losses.mean() if len(excess_losses) > 0 else np.nan

# ----------------------------
# 10. 输出结果
# ----------------------------
print("=== VaR/ES 回测结果（95%置信水平）===")
print(f"95% VaR: {var_95:.4f}")
print(f"违约次数: {n_violations} / {n_obs}, Hit rate: {hit_rate:.4f}")
print(f"Kupiec LR_uc: {LR_uc:.4f}, p-value: {p_value_uc:.4f}")
print(f"Christoffersen 独立性检验 LR_ind: {L_ind:.4f}, p-value: {p_value_ind:.4f}")
print(f"实际 ES 平均超额损失: {es_realized:.4f}")


=== VaR/ES 回测结果（95%置信水平）===
95% VaR: -6.2609
违约次数: 119 / 2083, Hit rate: 0.0571
Kupiec LR_uc: 2.1351, p-value: 0.1440
Christoffersen 独立性检验 LR_ind: 1.5634, p-value: 0.2112
实际 ES 平均超额损失: -9.6791


In [2]:
from google.colab import files
uploaded = files.upload()  # 选择文件

Saving TSLA1.xlsx to TSLA1.xlsx


In [20]:
###真实数据

import numpy as np
import pandas as pd
from scipy.stats import skewnorm, t
from scipy.optimize import brentq
from scipy.integrate import quad
from scipy.stats import chi2

# -----------------------------
# 1. 读取 TSLA 收盘价
# -----------------------------
df = pd.read_excel('TSLA1.xlsx')
close_prices = df["close"].dropna()
close_prices = close_prices[close_prices > 0]

# 对数收益率 (%)
returns = 100 * np.log(close_prices / close_prices.shift(1)).dropna().values
dates = df["date"].iloc[1:]

# -----------------------------
# 2. 复合分布参数（你的表格）
# -----------------------------
theta1, theta2 = -1.439, 1.171
xi_c, omega_c, alpha_c = 0.148, 1.762, -0.016
xi_tc, omega_tc, alpha_tc, nu_c = -0.014, 2.687, -0.033, 3.253

# -----------------------------
# 3. 定义 skew-t PDF 和 CDF（尾部）
# -----------------------------
def s_t_skew(x, xi, omega, alpha, nu):
    z = (x - xi)/omega
    t_pdf = t.pdf(z, df=nu)/omega
    t_cdf = t.cdf(alpha * z * np.sqrt((nu+1)/(nu+z**2)), df=nu+1)
    return 2 * t_pdf * t_cdf

def S_t_skew(x, xi, omega, alpha, nu):
    val, _ = quad(lambda z: s_t_skew(z, xi, omega, alpha, nu), -100, x)  # 积分下限取极端值
    return val

# -----------------------------
# 4. 中心 skew-normal PDF/CDF
# -----------------------------
def s_n(x):
    return skewnorm.pdf(x, a=alpha_c, loc=xi_c, scale=omega_c)

def S_n(x):
    return skewnorm.cdf(x, a=alpha_c, loc=xi_c, scale=omega_c)

# -----------------------------
# 5. 计算尾部权重 r1, r2
# -----------------------------
s_t_theta1 = s_t_skew(theta1, xi_tc, omega_tc, alpha_tc, nu_c)
s_t_theta2 = s_t_skew(theta2, xi_tc, omega_tc, alpha_tc, nu_c)
S_t_theta1 = S_t_skew(theta1, xi_tc, omega_tc, alpha_tc, nu_c)
S_t_theta2 = S_t_skew(theta2, xi_tc, omega_tc, alpha_tc, nu_c)
s_n_theta1 = s_n(theta1)
s_n_theta2 = s_n(theta2)
S_n_theta1 = S_n(theta1)
S_n_theta2 = S_n(theta2)

r1 = (s_t_theta2 * s_n_theta1 * S_t_theta1) / (
    s_t_theta1*s_t_theta2*(S_n_theta2-S_n_theta1) + s_t_theta2*s_n_theta1*S_t_theta1 + s_t_theta1*s_n_theta2*(1-S_t_theta2)
)
r2 = 1 - r1 * (s_t_theta1*(S_n_theta2-S_n_theta1) + S_t_theta1*s_n_theta1)/(S_t_theta1*s_n_theta1)

# -----------------------------
# 6. 复合分布 CDF
# -----------------------------
def F_comp(x):
    if x <= theta1:
        return r1 * S_t_skew(x, xi_tc, omega_tc, alpha_tc, nu_c) / S_t_theta1
    elif x < theta2:
        return r1 + (1 - r1 - r2) * (S_n(x) - S_n_theta1) / (S_n_theta2 - S_n_theta1)
    else:
        return (1-r2) + r2 * (S_t_skew(x, xi_tc, omega_tc, alpha_tc, nu_c) - S_t_theta2)/(1-S_t_theta2)

# -----------------------------
# 7. 计算 VaR_95
# -----------------------------
alpha = 0.05
VaR_95 = brentq(lambda x: F_comp(x) - alpha, -20, 20)

# -----------------------------
# 8. 计算 ES_95（左尾）
# -----------------------------
def integrand(x):
    if x <= theta1:
        return x * s_t_skew(x, xi_tc, omega_tc, alpha_tc, nu_c) / S_t_theta1
    elif x >= theta2:
        return x * s_t_skew(x, xi_tc, omega_tc, alpha_tc, nu_c) / (1 - S_t_theta2)
    else:
        return 0

ES_95, _ = quad(integrand, -50, VaR_95)

# -----------------------------
# 9. 回测指标
# -----------------------------
violations = (returns < VaR_95).astype(int)
T = len(returns)
x_violate = violations.sum()
p_hat = x_violate / T

# Kupiec LR_uc
LR_uc = -2 * ((T - x_violate) * np.log(1 - alpha) + x_violate * np.log(alpha) -
              ((T - x_violate) * np.log(1 - p_hat) + x_violate * np.log(p_hat)))
pval_uc = 1 - chi2.cdf(LR_uc, df=1)

# Christoffersen 独立性检验
I_lag = violations[:-1]
I_curr = violations[1:]
n00 = np.sum((I_lag==0)&(I_curr==0))
n01 = np.sum((I_lag==0)&(I_curr==1))
n10 = np.sum((I_lag==1)&(I_curr==0))
n11 = np.sum((I_lag==1)&(I_curr==1))
pi0 = n01/(n00+n01) if (n00+n01)>0 else 0
pi1 = n11/(n10+n11) if (n10+n11)>0 else 0
L0 = ((1-alpha)**(n00+n10))*(alpha**(n01+n11))
L1 = ((1-pi0)**n00 * pi0**n01) * ((1-pi1)**n10 * pi1**n11)
LR_ind = -2*np.log(L0/L1) if L1>0 else np.nan
pval_ind = 1 - chi2.cdf(LR_ind, df=1) if not np.isnan(LR_ind) else np.nan

# 平均短缺误差
ASE = np.mean(ES_95 - returns[violations==1]) if x_violate>0 else np.nan

# -----------------------------
# 10. 输出结果
# -----------------------------
results = pd.DataFrame({
    "Date": dates,
    "Return": returns,
    "VaR_95": VaR_95,
    "ES_95": ES_95,
    "Violation": violations
})

print(results.head(10))
print(f"\n95% VaR = {VaR_95:.4f}, ES = {ES_95:.4f}")
print(f"Kupiec LR_uc = {LR_uc:.4f}, p-value = {pval_uc:.4f}")
print(f"Christoffersen LR_ind = {LR_ind:.4f}, p-value = {pval_ind:.4f}")
print(f"Average Shortfall Error (ASE) = {ASE:.4f}")


          Date    Return   VaR_95     ES_95  Violation
1   2017-01-04  4.505469 -5.90003 -1.645331          0
2   2017-01-05 -0.105787 -5.90003 -1.645331          0
3   2017-01-06  0.991758 -5.90003 -1.645331          0
4   2017-01-09  0.986343 -5.90003 -1.645331          0
5   2017-01-10 -0.611517 -5.90003 -1.645331          0
6   2017-01-11 -0.060923 -5.90003 -1.645331          0
7   2017-01-12 -0.060960 -5.90003 -1.645331          0
8   2017-01-13  3.492459 -5.90003 -1.645331          0
9   2017-01-17 -0.916914 -5.90003 -1.645331          0
10  2017-01-18  1.173158 -5.90003 -1.645331          0

95% VaR = -5.9000, ES = -1.6453
Kupiec LR_uc = 0.0134, p-value = 0.9078
Christoffersen LR_ind = 2.6614, p-value = 0.1028
Average Shortfall Error (ASE) = 9.7146


In [35]:
#TSLA,实际数据（效果好）
import numpy as np
import pandas as pd
from scipy.stats import norm, t, chi2
from scipy.integrate import quad
from scipy.optimize import bisect
from math import log

# ----------------------------
# 1. 定义 Skew-Normal 和 Skew-t PDF
# ----------------------------
def skew_normal_pdf(x, xi, omega, alpha):
    z = (x - xi) / omega
    phi_z = norm.pdf(z)
    Phi_alpha_z = norm.cdf(alpha * z)
    return (2 / omega) * phi_z * Phi_alpha_z

def skew_t_pdf(x, xi_t, omega_t, alpha_t, nu):
    z = (x - xi_t) / omega_t
    t_pdf = t.pdf(z, df=nu)
    a = alpha_t * z * np.sqrt((nu + 1) / (nu + z**2))
    t_cdf = t.cdf(a, df=nu + 1)
    return (2 / omega_t) * t_pdf * t_cdf

# Skew-Normal CDF
def skew_normal_cdf(x, xi, omega, alpha):
    return quad(lambda t: skew_normal_pdf(t, xi, omega, alpha), -np.inf, x)[0]

# Skew-t CDF
def skew_t_cdf(x, xi_t, omega_t, alpha_t, nu):
    return quad(lambda t: skew_t_pdf(t, xi_t, omega_t, alpha_t, nu), -np.inf, x)[0]



# ----------------------------
# 5. 设置参数
# ----------------------------
theta1, theta2 = -1.439, 1.171
xi, omega, alpha = 0.148, 1.762, -0.016
xi_t, omega_t, alpha_t, nu = -0.014, 2.687, -0.033, 3.253

numerator_r1 = skew_t_pdf(theta2, xi_t, omega_t, alpha_t, nu) * skew_normal_pdf(theta1, xi, omega, alpha) * skew_t_cdf(theta1, xi_t, omega_t, alpha_t, nu)
denominator_r1 = skew_t_pdf(theta1, xi_t, omega_t, alpha_t, nu) * skew_t_pdf(theta2, xi_t, omega_t, alpha_t, nu) * \
    (skew_normal_cdf(theta2, xi, omega, alpha) - skew_normal_cdf(theta1, xi, omega, alpha)) + \
    skew_t_pdf(theta2, xi_t, omega_t, alpha_t, nu) * skew_t_cdf(theta1, xi_t, omega_t, alpha_t, nu) * skew_normal_pdf(theta1, xi, omega, alpha) + \
    skew_t_pdf(theta1, xi_t, omega_t, alpha_t, nu) * skew_normal_pdf(theta2, xi, omega, alpha) * (1 - skew_t_cdf(theta2, xi_t, omega_t, alpha_t, nu))

r_1 = numerator_r1 / denominator_r1

numerator_r2 = skew_t_pdf(theta1, xi_t, omega_t, alpha_t, nu) * (skew_normal_cdf(theta2, xi, omega, alpha) - skew_normal_cdf(theta1, xi, omega, alpha)) + \
    skew_t_cdf(theta1, xi_t, omega_t, alpha_t, nu) * skew_normal_pdf(theta1, xi, omega, alpha)
denominator_r2 = skew_t_cdf(theta1, xi_t, omega_t, alpha_t, nu) * skew_normal_pdf(theta1, xi, omega, alpha)

r_2 = 1 - r_1 * (numerator_r2 / denominator_r2)

print("r_1, r_2:", r_1, r_2)

a=skew_t_cdf(theta1, xi_t, omega_t, alpha_t, nu)
b=skew_normal_cdf(theta2, xi, omega, alpha) - skew_normal_cdf(theta1, xi, omega, alpha)
c=1 - skew_t_cdf(theta2, xi_t, omega_t, alpha_t, nu)

print(a,b,c)

# ----------------------------
# 2. 定义复合分布 PDF
# ----------------------------
def composite_pdf(x, xi, omega , alpha , xi_t , omega_t, alpha_t, nu, theta1 , theta2):
    numerator_r1 = skew_t_pdf(theta2, xi_t, omega_t, alpha_t, nu) * skew_normal_pdf(theta1, xi, omega, alpha) * skew_t_cdf(theta1, xi_t, omega_t, alpha_t, nu)
    denominator_r1 = skew_t_pdf(theta1, xi_t, omega_t, alpha_t, nu) * skew_t_pdf(theta2, xi_t, omega_t, alpha_t, nu) * \
    (skew_normal_cdf(theta2, xi, omega, alpha) - skew_normal_cdf(theta1, xi, omega, alpha)) + \
    skew_t_pdf(theta2, xi_t, omega_t, alpha_t, nu) * skew_t_cdf(theta1, xi_t, omega_t, alpha_t, nu) * skew_normal_pdf(theta1, xi, omega, alpha) + \
    skew_t_pdf(theta1, xi_t, omega_t, alpha_t, nu) * skew_normal_pdf(theta2, xi, omega, alpha) * (1 - skew_t_cdf(theta2, xi_t, omega_t, alpha_t, nu))

    r_1 = numerator_r1 / denominator_r1

    numerator_r2 = skew_t_pdf(theta1, xi_t, omega_t, alpha_t, nu) * (skew_normal_cdf(theta2, xi, omega, alpha) - skew_normal_cdf(theta1, xi, omega, alpha)) + \
    skew_t_cdf(theta1, xi_t, omega_t, alpha_t, nu) * skew_normal_pdf(theta1, xi, omega, alpha)
    denominator_r2 = skew_t_cdf(theta1, xi_t, omega_t, alpha_t, nu) * skew_normal_pdf(theta1, xi, omega, alpha)

    r_2 = 1 - r_1 * (numerator_r2 / denominator_r2)

    if x <= theta1:
        return (r_1 / skew_t_cdf(theta1, xi_t, omega_t, alpha_t, nu)) * skew_t_pdf(x, xi_t, omega_t, alpha_t, nu)
    elif theta1 < x < theta2:
        return ((1 - r_1 - r_2) / (skew_normal_cdf(theta2, xi, omega, alpha) - skew_normal_cdf(theta1, xi, omega, alpha))) * skew_normal_pdf(x, xi, omega, alpha)
    else:
        return (r_2 / (1 - skew_t_cdf(theta2, xi_t, omega_t, alpha_t, nu))) * skew_t_pdf(x, xi_t, omega_t, alpha_t, nu)

# ----------------------------
# 3. 定义复合分布 CDF（数值积分）
# ----------------------------
from scipy.optimize import brentq

def composite_cdf_exact(x, xi, omega, alpha, xi_t, omega_t, alpha_t, nu, theta1, theta2, r1, r2):
    # 分母标准化
    S_t_theta1 = skew_t_cdf(theta1, xi_t, omega_t, alpha_t, nu)
    S_t_theta2 = skew_t_cdf(theta2, xi_t, omega_t, alpha_t, nu)
    S_n_theta1 = skew_normal_cdf(theta1, xi, omega, alpha)
    S_n_theta2 = skew_normal_cdf(theta2, xi, omega, alpha)

    # 左段
    if x <= theta1:
        integral, _ = quad(lambda z: skew_t_pdf(z, xi_t, omega_t, alpha_t, nu), -np.inf, x)
        return r1 * integral / S_t_theta1

    # 中段
    elif theta1 < x < theta2:
        integral, _ = quad(lambda z: skew_normal_pdf(z, xi, omega, alpha), theta1, x)
        return r1 + (1 - r1 - r2) * integral / (S_n_theta2 - S_n_theta1)

    # 右段
    else:
        integral, _ = quad(lambda z: skew_t_pdf(z, xi_t, omega_t, alpha_t, nu), theta2, x)
        return (1 - r2) + r2 * integral / (1 - S_t_theta2)

# -----------------------------
# 6. 读取 TSLA 收盘价
# -----------------------------
df = pd.read_excel('TSLA1.xlsx')
close_prices = df["close"].dropna()
close_prices = close_prices[close_prices > 0]

# 对数收益率 (%)
returns = 100 * np.log(close_prices / close_prices.shift(1)).dropna().values

# -----------------------------
# 5. 计算 95% VaR
# -----------------------------
alpha_level = 0.05
var_95 = brentq(
    lambda x: composite_cdf_exact(x, xi, omega, alpha,
                                  xi_t, omega_t, alpha_t, nu,
                                  theta1, theta2, r_1, r_2) - alpha_level,
    -50, 50
)

# -----------------------------
# 6. 违约次数 & Hit rate
# -----------------------------
violations = returns < var_95
n_violations = violations.sum()
n_obs = len(returns)
hit_rate = n_violations / n_obs

# -----------------------------
# 7. Kupiec LR_uc
# -----------------------------
eps = 1e-16
p = np.clip(hit_rate, eps, 1-eps)
LR_uc = -2 * ( (n_obs-n_violations)*log(1-alpha_level) + n_violations*log(alpha_level) -
               (n_obs-n_violations)*log(1-p) - n_violations*log(p) )
p_value_uc = 1 - chi2.cdf(LR_uc, df=1)

# -----------------------------
# 8. Christoffersen 独立性检验
# -----------------------------
violations_int = violations.astype(int)
N00 = N01 = N10 = N11 = 0
for t in range(1, n_obs):
    if violations_int[t-1] == 0 and violations_int[t] == 0:
        N00 += 1
    elif violations_int[t-1] == 0 and violations_int[t] == 1:
        N01 += 1
    elif violations_int[t-1] == 1 and violations_int[t] == 0:
        N10 += 1
    else:
        N11 += 1

pi01 = N01 / (N00 + N01) if (N00 + N01) > 0 else 0
pi11 = N11 / (N10 + N11) if (N10 + N11) > 0 else 0
pi = (N01 + N11) / (N00 + N01 + N10 + N11)
pi01 = np.clip(pi01, eps, 1-eps)
pi11 = np.clip(pi11, eps, 1-eps)
pi = np.clip(pi, eps, 1-eps)
L_ind = 2 * ((N00 * log(1-pi01) + N01 * log(pi01) + N10 * log(1-pi11) + N11 * log(pi11)) -
             ((N00 + N10) * log(1-pi) + (N01 + N11) * log(pi)))
p_value_ind = 1 - chi2.cdf(L_ind, df=1)

# -----------------------------
# 9. 实际 ES
# -----------------------------
excess_losses = returns[returns < var_95]
es_realized = excess_losses.mean() if len(excess_losses) > 0 else np.nan

# -----------------------------
# 10. 输出回测结果
# -----------------------------
print("=== VaR/ES 回测结果（95%置信水平）===")
print(f"95% VaR: {var_95:.4f}")
print(f"违约次数: {n_violations} / {n_obs}, Hit rate: {hit_rate:.4f}")
print(f"Kupiec LR_uc: {LR_uc:.4f}, p-value: {p_value_uc:.4f}")
print(f"Christoffersen 独立性检验 LR_ind: {L_ind:.4f}, p-value: {p_value_ind:.4f}")
print(f"实际 ES 平均超额损失: {es_realized:.4f}")

r_1, r_2: 0.28437444289679037 0.3548862345451448
0.3241601917327313 0.53627090051363 0.3338893724143177


KeyboardInterrupt: 